In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# List of S&P 100 tickers
sp100_tickers = [
    "AAPL", "ABBV", "ABT", "ACN", "AIG", "ALL", "AMGN", "AMT", "AMZN", "AVGO", "AXP", "BA", "BAC", "BK",
    "BKNG", "BLK", "BMY", "BRK-B", "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP", "COST", "CRM", "CSCO",
    "CVS", "CVX", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD", "GM", "GOOGL",
    "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMB", "KO", "LIN", "LLY", "LMT", "LOW", "MA",
    "MCD", "MDLZ", "MDT", "MET", "META", "MMM", "MO", "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE", "NVDA",
    "ORCL", "PEP", "PFE", "PG", "PM", "PYPL", "QCOM", "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO",
    "TMUS", "TXN", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WBA", "WFC", "WMT", "XOM"
]

def get_free_cash_flow(ticker):
    try:
        stock = yf.Ticker(ticker)
        cash_flow = stock.cashflow.loc['Total Cash From Operating Activities'] - stock.cashflow.loc['Capital Expenditures']
        return cash_flow
    except Exception as e:
        print(f"Error getting FCF for {ticker}: {e}")
        return None

def get_market_cap(ticker, start_date, end_date):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        market_cap = hist['Close'] * stock.info['sharesOutstanding']
        return market_cap
    except Exception as e:
        print(f"Error getting market cap for {ticker}: {e}")
        return None

def calculate_rolling_fcf_yield(ticker, start_date, end_date):
    fcf = get_free_cash_flow(ticker)
    market_cap = get_market_cap(ticker, start_date, end_date)

    if fcf is None or market_cap is None:
        return None
    
    try:
        fcf_monthly = fcf.resample('M').mean()
        fcf_yield = fcf_monthly / market_cap.resample('M').mean()
        rolling_fcf_yield = fcf_yield.rolling(window=12).mean().dropna()
        return rolling_fcf_yield
    except Exception as e:
        print(f"Error calculating rolling FCF yield for {ticker}: {e}")
        return None

start_date = "2010-01-01"
end_date = "2020-01-01"

fcf_yields = {}
for ticker in sp100_tickers:
    fcf_yield = calculate_rolling_fcf_yield(ticker, start_date, end_date)
    if fcf_yield is not None:
        fcf_yields[ticker] = fcf_yield

fcf_yields_df = pd.DataFrame(fcf_yields)

# Rank stocks by FCF Yield at each time point
ranked_stocks = fcf_yields_df.rank(axis=1, ascending=False)
top_10_percent_stocks = ranked_stocks.apply(lambda row: row[row <= len(row) * 0.1].index.tolist(), axis=1)

# Initialize portfolio and variables
initial_cash = 100000
cash = initial_cash
portfolio = {}
portfolio_values = []

rebalance_dates = pd.date_range(start=start_date, end=end_date, freq='6M')

for date in rebalance_dates:
    if date not in top_10_percent_stocks.index:
        continue
    current_stocks = top_10_percent_stocks.loc[date]
    
    # Sell all stocks
    for stock in list(portfolio.keys()):
        price = yf.Ticker(stock).history(start=date, end=date)['Close'].values[0]
        cash += portfolio[stock] * price
        del portfolio[stock]
    
    # Buy new stocks
    allocation = cash / len(current_stocks)
    for stock in current_stocks:
        price = yf.Ticker(stock).history(start=date, end=date)['Close'].values[0]
        shares = allocation // price
        portfolio[stock] = shares
        cash -= shares * price
    
    # Record portfolio value
    portfolio_value = cash + sum([portfolio[stock] * yf.Ticker(stock).history(start=date, end=date)['Close'].values[0] for stock in portfolio])
    portfolio_values.append(portfolio_value)

# Convert to DataFrame for easier plotting
portfolio_values_df = pd.DataFrame(portfolio_values, index=rebalance_dates, columns=['Portfolio Value'])

# Plot Portfolio Value Over Time
portfolio_values_df.plot(figsize=(12, 6))
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.show()


Error getting FCF for AAPL: 'Total Cash From Operating Activities'
Error getting market cap for AAPL: 401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
Error getting FCF for ABBV: 'Total Cash From Operating Activities'
Error getting market cap for ABBV: 401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABBV?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
Error getting FCF for ABT: 'Total Cash From Operating Activities'
Error getting market cap for ABT: 401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABT?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
Error getting FCF for ACN: 'Total Cash From Op

KeyboardInterrupt: 